# Notebook #3: Exploratory Data Analyses, Statistical Testing, and Time Series Analysis
The `rhino_health` Python library is a robust tool designed to support a wide array of statistical and epidemiological analyses over federated data sets, crucial in the realm of healthcare and medical research. In this notebook, we demonstrate the ability to perform analyses over multiple distributed datasets. More specifically, we'll perform an analysis of data related to pneumonia outcomes and perform both exploratory data analyses to support our machine learning project as well as hypothesis testing for a traditional biostatistics analysis.

#### Import Rhino's Metrics Module

The `rhino_health.lib.metrics` module in the Rhino Health library is a comprehensive suite designed for diverse statistical and epidemiological analyses of healthcare data. This module is divided into several submodules, each targeting specific types of metrics and analyses.

In the following code block, we'll import some basic functions like Mean() and Count() and authenticate to the Rhino cloud using your specific credentials.

### Install the Rhino Health Python SDK, Load All Necessary Libraries and Login to the Rhino FCP

In [ ]:
import getpass
from pprint import pprint
import rhino_health as rh

from rhino_health.lib.metrics.basic import Sum, Count, Mean,StandardDeviation
from rhino_health.lib.metrics.aggregate_metrics.aggregation_service import get_cloud_aggregated_metric_data

In [ ]:
my_username = "FCP_LOGIN_EMAIL" # Replace this with the email you use to log into Rhino Health
session = rh.login(username=my_username, password=getpass.getpass())

### Load the Datasets
We'll use our SDK to identify the relevant Datasets that we'd like to perform exploratory analyses on. It is **critical to understand that the Datasets must have the same Data Schema in order to generate statistics on multiple Datasets simultaneously.**

<Include screenshot of the data schema>

In [ ]:
#Replace with your Project and Dataset names. Raw data and harmonized data
project = session.project.get_project_by_name("YOUR_PROJECT_NAME")

cxr_datasets = [
    project.get_dataset_by_name("mimic_cxr_dev"), # Replace Me
    project.get_dataset_by_name("mimic_cxr_hco"), # Replace Me
] 

### An Introduction to Federated Metrics in the Rhino SDK. 

The Rhino Federated Computing Platform allows you to quickly and securely calculate metrics using federated computing across multiple sites. When creating a **Metric** object is necessary to specify a 1-tailed t-test, chi-square test, or arbitrary correlation statistic. Additionally, you can **configure** your Metric object with helper functions like grouping preferences and data filters.

The Rhino SDK returns the results of an executed Federated Metric query in a **MetricResponse** object. The MetricResponse object is a Python class that includes the specific outcome values in the 'output' attribute, such as statistical measures, and details about the metric configuration ('metric_configuration_dict').

### Exploratory Data Analysis

In the rest of this notebook, we'll analyze the federated data that we've prepared in the preceding notebooks. The data will be aggregated across two sites.

### Defining a simple metric without a filter:

We'll define the simplest metric possible -  a simple count of the number of rows across both of our Datasets: 

In [ ]:
# Count the number of entries in the dataset
pneumonia_count_response = session.project.aggregate_dataset_metric( 
    dataset_uids=[str(dataset.uid) for dataset in cxr_datasets], # list containing relevant Datasets
    metric_configuration=Count(variable="Pneumonia") # Metric configuration
) 

pneumonia_count = pneumonia_count_response.output
print(f"Entries in Dataset: {pneumonia_count}")

### Adding a filter to our metric
The `data_filters` parameter enables you to refine your analysis by setting conditions and filter the output by certain criteria. We'll now filter our `Count()` variable by a value; in this case, pneumona cases are identified by the `pneumonia` value of 1, and thus we'll add `pneumonia` as a `filter_column` and `1` as a `filter_value`.

In [ ]:
# Count the number of people with pneumonia and the number without pneumonia
pneumonia_count_configuration = Count(variable={"data_column": "Pneumonia", 
                                         "filter_column": "Pneumonia",  
                                         "filter_value": 1})

pneumonia_count_response = session.project.aggregate_dataset_metric(
    dataset_uids=[str(dataset.uid) for dataset in cxr_datasets],
    metric_configuration=pneumonia_count_configuration)
                                         
pneumonia_count = pneumonia_count_response.output

print(f"Pneumonia Cases in Dataset: {pneumonia_count}")

#### Adding a grouping mechanism to our metric
In addition to the `data_filter parameter`, we can also add a `group_by` parameter allows you to organize metrics based on specific categorical variables. In this example, we'll calculate the mean age across our two Datasets using the gender column in our data.

In [ ]:
# Get median age of the aggregated Datasets
median_age_response = session.project.aggregate_dataset_metric(
    dataset_uids=[str(dataset.uid) for dataset in cxr_datasets],
    metric_configuration=Median(variable="age",
                                group_by="gender"),
)

median_age = median_age_response.output["age"]
print(f"Median Age: {median_age}")

#### Contingency Tables (aka two-by-two tables)
The `TwoByTwoTable` metric facilitates the creation of a two-by-two contingency table, enabling you to analyze the relationships between variables. Here is an example for generating a two by two table metric for the columns "exposed" and "detected" in the data:

Lets examine the count of men and women with and without our outcome of interest (pneumonia). We can use Rhino's two-by-two table functions to generate simple contingency tables. 

The table results are also stored in a response object, that can be parsed into a pandas data frame in order to view the results as a table:

In [ ]:
from rhino_health.lib.metrics import Count, FilterType, Mean, StandardDeviation, RocAuc
from rhino_health.lib.metrics.epidemiology.two_by_two_table_based_metrics import *

# Calculate TBTT
tbtt = TwoByTwoTable(
            variable="subject_id",
        detected_column_name="gender",
    exposed_column_name="pneumonia",
)


table_result = project.aggregate_dataset_metric([str(dataset.uid) for dataset in cxr_datasets], tbtt).output
pd.DataFrame(table_result.as_table())

Interestingly, we can see that our Dataset is extremely skewed towards women with pneumonia. 

#### Odds Ratio:
We can configure an odds ratio metric using the same configuration and execution pattern that we defined above for the median statistic. The Odds metric calculates the odds of an event occurring rather than not occuring, and can be generated like so: 

In [ ]:
odds_ratio_config = OddsRatio(
    variable="subject_id",
    exposed_column_name="gender",
    detected_column_name="pneumonia",
)

session.project.aggregate_dataset_metric([str(dataset.uid) for dataset in cxr_datasets], odds_ratio_config).output

#### Prevalence & Incidence

The Prevalence metric calculates the proportion of individuals who have or develop a specific condition over a specified time range, whereas the Incidence describes the occurrence of new cases over a period of time. In this example, the prevalence and incidence of pneumonia is calculated between the given specific time range. Note that the column representing the time data should contain time in UTC format.

In [ ]:
from rhino_health.lib.metrics import Prevalence, Incidence 

prevalence_config = Prevalence(variable="id", 
                               time_column_name="Time Pneumonia", 
                               detected_column_name="Pneumonia", 
                               start_time="2023-02-02T07:07:48Z", 
                               end_time="2023-06-10T11:24:43Z", ) 
prevalence_results = session.project.aggregate_dataset_metric(cxr_datasets, prevalence_config)


incidence_config = Incidence( variable="id", 
                             time_column_name="Time Pneumonia", 
                             detected_column_name="Pneumonia", 
                             start_time="2023-02-02T07:07:48Z", 
                             end_time="2023-06-10T11:24:43Z", ) 
incidence_results = session.project.aggregate_dataset_metric(cxr_datasets, incidence_config)

## Statistical Testing

#### Chi-Square Test

The Chi-Square test is employed to assess the independence between two categorical variables. In this example, we examine the association between the occurrence of pneumonia and gender across different Datasets. The result includes the Chi-Square statistic, p-value, and degrees of freedom.

In [ ]:
from rhino_health.lib.metrics.statistics_tests import ChiSquare

chi_square_config = ChiSquare(variable="id", variable_1="Pneumonia", variable_2="Gender")

result = project.aggregate_dataset_metric(cxr_datasets, chi_square_config)

#### T-Test

The T-Test is utilized to determine if there is a significant difference between the means of two groups. The implemented method is the Welch test, that does not assume equality of variance. The result includes the T-Test statistic, p-value, and degrees of freedom.

In [ ]:
from rhino_health.lib.metrics.statistics_tests import TTest

t_test_config = TTest(numeric_variable="Height", categorical_variable="Gender")

t_test_result = project.aggregate_dataset_metric(cxr_datasets, t_test_config)

#### One-Way ANOVA

The One-Way ANOVA (Analysis of Variance) is applied to assess whether there are any statistically significant differences between the means of three or more independent groups. In this example, we examine the relationship between inflammation level and height. The result will contain the following calculated values: anova statistic value, p value, dfc, dfe, dft, MSC, MSE, SSC, SSE, SST. 

In [ ]:
from rhino_health.lib.metrics.statistics_tests import OneWayANOVA

anova_config = OneWayANOVA(variable="id", numeric_variable="Height", categorical_variable="Inflammation Level")

anova_result = project.aggregate_dataset_metric(cxr_datasets, anova_config)

## Time Series Analysis
The Rhino SDK also has the ability to perform Kaplan Meier analyses. The Kaplan-Meier Metric is a powerful tool for analyzing time-to-event data, such as patient survival rates.  The KaplanMeier() function returns the k-percentile of entries for a specified variable. 

For configuring the basic Kaplan-Meier metric, you can set up the metric and retrieve results as follows:

In [ ]:
from rhino_health.lib.metrics import KaplanMeier

# Set the time and event variables
time_variable = "time_column_name"
event_variable = "event_column_name"

# Create a KaplanMeier instance
metric_configuration = KaplanMeier(time_variable=time_variable, event_variable=event_variable)

# Retrieve results for your Project and Datasets
results = project.aggregate_dataset_metric(dataset_uids=[str(dataset.uid) for dataset in cxr_datasets], metric_configuration=metric_configuration)

#### Working with Kaplan-Meier Metric Results
Extracting Time and Events Vector

The Kaplan-Meier Metric in the Rhino Health Platform provides results that allow you to analyze time-to-event data, create survival models, and visualize Kaplan-Meier curves. 

The results of the Kaplan-Meier Metric are stored in a KaplanMeierMetricResponse object with an "output" attribute that contains time and event vectors. Access these vectors as follows:

In [ ]:
# Accessing the vectors using the names of the original time and event data columns
# For non grouped results
time_vector = results.output["time_column_name"]
event_vector = results.output["event_column_name"]

# For grouped results, where the group of interest is "1"
time_vector_group_1 = results.output["1"]["time_column_name"]
event_vector_group_1 = results.output["1"]["event_column_name"]

By obtaining these vectors, you can proceed to create a survival model and gain more insights from your Kaplan-Meier data in any way desired. 


### Creating a Survival Model

The Rhino Health Platform SDK provides a convenient way to obtain the survival model object, which allows you to explore detailed Kaplan-Meier analysis. The object is a SurvFuncRight object from the statsmodels library:m

In [ ]:
# For non grouped results
survival_model = results.surv_func_right_model()

# For grouped results, get the survival model where the group of interest is "1"
group = "1"
survival_model = results.surv_func_right_model(group=group)

# Access various properties of the survival model
median_time = survival_model.quantile(0.5)  # Median survival time
cumulative_hazard = survival_model.cumulative_hazard_at_times([100, 200, 300])  # Cumulative hazard at specific times
print(survival_model.summary())

Note that to use this feature, you need to have the statsmodels library installed in your Python environment. If you haven't installed it yet, you can do so using `pip`:

In [ ]:
pip install statsmodels

### Plotting Kaplan-Meier Curves

Visualizing Kaplan-Meier curves is a way to gain insights into your survival data. The Rhino Health Platform SDK KaplanMeierMetricResults object can be used to plot these curves. Using matplotlib.pyplot library is a convenient way for that:

In [ ]:
import matplotlib.pyplot as plt

# Accessing time and event vectors
time_vector = results.output["time_column_name"]
event_vector = results.output["event_column_name"]

# Plot Kaplan-Meier survival curve
plt.figure(figsize=(10, 6))
plt.step(time_vector, event_vector, where='post', label="model 1")
plt.legend(loc="upper left")
plt.title("Kaplan-Meier Survival Curve")
plt.xlabel("Time")
plt.ylabel("Survival Probability")
plt.show()

#### Differential Privacy for Kaplan-Meier Metric

Differential privacy is a technique used in the FCP to protect patient data by adding noise to query results. Like all FCP metrics, the Kaplan-Meier metric supports differential privacy, and you can configure the privacy enforcement level in your project settings. The default privacy level is "Medium," but you can select from "None," "Low," "Medium," or "High" according to your project's privacy requirements, whereas:

None - No noise is added to any of the data.
Low, Medium - Noise is partially added to the data. Times that less than k (anonymity threshold) events occur are aggregated and averaged across events occurring in adjacent times, and noise is then added to them.
High - Noise is added to all of the time data.  

To learn more about configuring differential privacy settings, please refer to the project settings documentation.